# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with `mongoimport --db uk_food --collection establishments --file Resources/establishments.json --jsonArray`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in 'uk_food' database:", collections)

Collections in 'uk_food' database: ['establishments']


In [6]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in 'uk_food' database:", collections)

Collections in 'uk_food' database: ['establishments']


In [7]:
# review a document in the establishments collection
document = db.establishments.find_one()
pprint(document)

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6707eeb021bc4028575cd544'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",  
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511", 
    "LocalAuthorityName": "Greenwich", 
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "", 
        "Structural": "", 
        "ConfidenceInManagement": ""
    }, 
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": '',
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [10]:
# Insert the new restaurant into the collection
insert_restaurant = db.establishments.insert_one(new_restaurant)

In [11]:
# Check that the new restaurant was inserted
# Confirming the insertion by printing the inserted ID and restaurant name
print("New restaurant inserted:", new_restaurant["BusinessName"])

New restaurant inserted: Penang Flavours


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}
business_type_result = db.establishments.find_one(business_type_query, {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1})

# Printing the result
print(business_type_result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID

# Defining the restaurant to update
restaurant_to_update = "Penang Flavours"

# Defining the fields to update
update_field = {
    "BusinessTypeID": 1
}

# Updating the defined fields
update_result = db.establishments.update_one(
    {"BusinessName": restaurant_to_update},  # BusinessName
    {"$set": update_field}  # Updation
)

In [14]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print(f"Updated '{restaurant_to_update}' restaurant with BusinessTypeID. '{update_field}'")
else:
    print("No changes made or restaurant not found.")

Updated 'Penang Flavours' restaurant with BusinessTypeID. '{'BusinessTypeID': 1}'


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments_in_dover = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"There are {establishments_in_dover} establishments in Dover")

There are 994 establishments in Dover


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"

# Defining the establishments to delete
establishments_to_delete = "Dover"

# Delete all documents with the specified BusinessName
delete_result = db.establishments.delete_many({"LocalAuthorityName": establishments_to_delete})

# Check the result of the delete operation
if delete_result.deleted_count > 0:
    print(f"Deleted {delete_result.deleted_count} establishments with LocalAuthorityName '{establishments_to_delete}'.")
else:
    print("No restaurants found with the specified BusinessName to delete.")

Deleted 994 establishments with LocalAuthorityName 'Dover'.


In [17]:
# Check if any remaining documents include Dover
remaining_dover_documents = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

if remaining_dover_documents > 0:
    print(f"There are still {remaining_dover_documents} establishments with LocalAuthorityName 'Dover'.")
else:
    print("No remaining documents include 'Dover'")

No remaining documents include 'Dover'


In [18]:
# Check that other documents remain with 'find_one'
remaining_document = db.establishments.find_one()

if remaining_document:
    print("Other establishments remain in the database.")
    print("Example document:")
    print(remaining_document)
else:
    print("No documents remain in the database.")

Other establishments remain in the database.
Example document:
{'_id': ObjectId('6707eeb021bc4028575cd82b'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude

# Updatinh the longitude and latitude fields from strings to decimal numbers
update_coordinates = db.establishments.update_many(
    {
        "geocode.longitude": {"$type": "string"},
        "geocode.latitude": {"$type": "string"}
    },  # Finding documents where longitude and latitude are strings
    [
        {
            "$set": {
                "geocode.longitude": {
                    "$cond": {
                        "if": {"$eq": ["$geocode.longitude", ""]},  # Check if the value is an empty string
                        "then": None,  # Set to null if empty
                        "else": {"$toDouble": "$geocode.longitude"}  # Convert to double otherwise
                    }
                },
                "geocode.latitude": {
                    "$cond": {
                        "if": {"$eq": ["$geocode.latitude", ""]},  # Check if the value is an empty string
                        "then": None,  # Set to null if empty
                        "else": {"$toDouble": "$geocode.latitude"}  # Convert to double otherwise
                    }
                }
            }
        }
    ]
)

# Check the result of the update operation
if update_coordinates.modified_count > 0:
    print(f"Updated {update_coordinates.modified_count} establishments to convert longitude and latitude to decimal numbers.")
else:
    print("No changes made or no establishments with longitude and latitude as strings found.")


Updated 38786 establishments to convert longitude and latitude to decimal numbers.


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
# Update the RatingValue field
update_ratings = db.establishments.update_many(
    {
        "RatingValue": {"$in": [0, "0", "1", "2", "3", "4", "5"]}  # Select records with valid rating values
    },
    [
        {
            "$set": {
                "RatingValue": {
                    "$cond": {
                        "if": {"$eq": ["$RatingValue", None]},  # If it's None, leave it as None
                        "then": None,
                        "else": {"$toInt": "$RatingValue"}  # Convert to integer
                    }
                }
            }
        }
    ]
)

# Get the count of documents updated
if update_ratings.modified_count > 0:
    print(f"Updated {update_ratings.modified_count} establishments to convert RatingValue to integers where applicable.")
else:
    print("No changes made or no valid RatingValue found for conversion.")

Updated 34694 establishments to convert RatingValue to integers where applicable.


In [22]:
# Check if longitude, latitude, and RatingValue are numbers (decimals or integers)
coordinates_and_rating_check = db.establishments.aggregate([
    {
        "$project": {
            "longitude_is_number": {
                "$in": [{"$type": "$geocode.longitude"}, ["double", "int"]]
            },
            "latitude_is_number": {
                "$in": [{"$type": "$geocode.latitude"}, ["double", "int"]]
            },
            "RatingValue_is_number": {
                "$in": [{"$type": "$RatingValue"}, ["int"]]
            }
        }
    },
    {
        "$group": {
            "_id": None,
            "valid_coordinates_count": {
                "$sum": {
                    "$cond": [
                        { "$and": ["$longitude_is_number", "$latitude_is_number"] },
                        1,
                        0
                    ]
                }
            },
            "valid_rating_count": {
                "$sum": {
                    "$cond": [
                        "$RatingValue_is_number",
                        1,
                        0
                    ]
                }
            },
            "total_documents": { "$sum": 1 }
        }
    }
])

# Output the results
for result in coordinates_and_rating_check:
    valid_coordinates_count = result['valid_coordinates_count']
    valid_rating_count = result['valid_rating_count']
    total_documents = result['total_documents']
    
    # Calculate the difference for invalid coordinates and RatingValue
    invalid_coordinates_count = total_documents - valid_coordinates_count
    invalid_rating_count = total_documents - valid_rating_count
    
    print(f"Total documents checked: {total_documents}")
    print(f"Documents with valid coordinates (longitude & latitude as numbers): {valid_coordinates_count}")
    print(f"Documents with valid RatingValue (as integer): {valid_rating_count}")
    
    # Print the difference attributed to None or blank values
    print(f"Documents with invalid coordinates (None or blank): {invalid_coordinates_count}")
    print(f"Documents with invalid RatingValue (None or blank): {invalid_rating_count}")


Total documents checked: 38786
Documents with valid coordinates (longitude & latitude as numbers): 38786
Documents with valid RatingValue (as integer): 34694
Documents with invalid coordinates (None or blank): 0
Documents with invalid RatingValue (None or blank): 4092
